# Demonstrate Interaction of pandas Dataframes and omnikeeper

In [107]:

from omnikeeper_client.functions import get_access_token, create_graphql_client, create_logger, execute_graphql
from omnikeeper_client import simple_traits
import os
import uuid

define some nameings

In [119]:
trait_name = "python_client_demo.test"
layer_name = "testlayer"

as usual create our client

In [121]:
config = dict(
    omnikeeper_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)
access_token = get_access_token(config)
client = create_graphql_client("%s/graphql" % config['omnikeeper_url'], access_token)

ensure trait ``python_client_demo.test`` exists, define a function to interact with traits - this is obviosly somthing to be pushed to api/sdk ;-)

In [74]:
from gql import gql, Client

def upsert_test_trait(client: Client):
    # create trait
    execute_graphql(client, gql("""
    mutation {
      manage_upsertRecursiveTrait(
        trait: {
          id: "python_client_demo.test"
          requiredAttributes: [
            {
              identifier:"id",
              template: {
                name:"test.id",
                type:INTEGER,
                isArray:false,
                isID:false,
                valueConstraints:[]
              }
            },
            {
              identifier:"array",
              template: {
                name:"test.array",
                type:TEXT,
                isArray:true,
                isID:false,
                valueConstraints:[]
              }
            },
            {
              identifier:"some_key",
              template: {
                name:"test.mapped_some_key",
                type:TEXT,
                isArray:false,
                isID:false,
                valueConstraints:[]
              }
            }

          ]
          optionalAttributes: []
          optionalRelations: []
          requiredTraits: []
        }
      ) {
        id
      }
    }
    """))


upsert_test_trait(client)    

create a test DataFrame

In [122]:
import pandas as pd

df_init = pd.DataFrame.from_records([
    {"id": 1, "array": ["a", "b"], "some_key": "Value 1"},
    {"id": 3, "array": ["c", "d"], "some_key": "another Value"},
])

df_init

,id,array,some_key
0,1,"[a, b]",Value 1
1,3,"[c, d]",another Value


now write data to layer ``testlayer`` (ensure it exists from previuos samples)

In [123]:
# insert that new data, this will also delete all old trait entities, if there are any
simple_traits.bulk_replace(client, trait_name=trait_name, input=df_init, id_attributes=["id"], id_relations=[], write_layer=layer_name, filter={})

True

load dataframe again

In [124]:
# get that data back out from omnikeeper
# what we get back has an index that's the CIID of the corresponding CI

df_work = simple_traits.get_all(client, trait_name=trait_name, layers=[layer_name])
df_work

,id,array,some_key
ciid,,,
3fd8cba5-f35b-4e5c-a88f-f8f1645e3445,3,"[c, d]",another Value
7cecf6f8-c5a8-476c-8d20-f46f8dd2657e,1,"[a, b]",Value 1


modify our dataframe

In [125]:
# change things in the returned data
# change single value of existing data
df_work.iat[0, df_work.columns.get_loc("array")] = ["a", "b", "z"]
# add a new row, using our own created CIID
df_work = pd.concat([df_work, pd.DataFrame([{"id": 4, "array": ["x"], "some_key": "the new guy"}], index=[str(uuid.uuid4())])])
# drop a row
df_work.drop([df_work.index[1]], inplace = True)

df_work

,id,array,some_key
3fd8cba5-f35b-4e5c-a88f-f8f1645e3445,3,"[a, b, z]",another Value
5e45e9b6-b9bc-4086-9ac8-4b24d3dea25e,4,[x],the new guy


write it back

In [127]:
simple_traits.set_all(client, trait_name=trait_name, input=df_work, write_layer=layer_name)


True

now compare it

In [128]:
df_final = simple_traits.get_all(client, trait_name=trait_name, layers=[layer_name])

print(f"Dataframes equal?: {df_final.equals(df_work)}") 

df_final


Dataframes equal?: True


,id,array,some_key
ciid,,,
3fd8cba5-f35b-4e5c-a88f-f8f1645e3445,3,"[a, b, z]",another Value
5e45e9b6-b9bc-4086-9ac8-4b24d3dea25e,4,[x],the new guy
